In [1]:
import pandas as pd
import numpy as np

from datetime import date
from datetime import datetime
import datetime

import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

from sklearn.neighbors import NearestNeighbors
from itertools import combinations

datos = pd.read_csv('C:/Users/Paula/Desktop/TFG/rtve-observations-episodes-2019-11-01.zip')


In [2]:
datos.head()

,id,date,metric,value,user,referer,os_family,os_version,browser_family,browser_version,...,podcast_permalink,podcast_topic,entries_processed,hostname,city,region,country,location,zip_code,time_zone
0,1a50eda4-26b1-47df-949d-4827665b79e3,2019-11-01T00:56:36.000,Número de descargas de tipo 206,1,6624207285322504776,NaN,Android,6.0.1,Android,6.0.1,...,RADIO-NACIONAL,Informativos,4,NaN,Palencia,Castilla y León,España,"42.0021, -4.5237",34004,Europe/Madrid
1,da19819a-3784-41ef-9ecd-67d6e4be37aa,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1,1093782487219863411,NaN,Android,8.0.0,Android,8.0.0,...,RADIO-NACIONAL,Informativos,21,NaN,Madrid,Madrid,España,"40.4143, -3.7016",28021,Europe/Madrid
2,a60f41aa-6b3d-4fe9-bf9b-6311c57abfc1,2019-11-01T00:49:33.000,Número de descargas de tipo 206,1,-3042089131472286197,RTVE,Windows,7,Chrome,77.0.3865,...,RADIO-CLASICA,Música,7,NaN,San Sebastián Tutla,Oaxaca,México,"17.0645, -96.6705",71242,America/Mexico_City
3,e85a8032-abad-48b4-9bd2-2765a4b730f4,2019-11-01T01:27:12.000,Número de descargas de tipo 206,1,-5995978442346962145,NaN,Android,9,Other,NaN,...,RADIO-NACIONAL,Entretenimiento,15,NaN,La Matanza de Acentejo,Canarias,España,"28.4524, -16.4472",38370,Atlantic/Canary
4,f3dabef8-f024-4f49-a04e-db6bbc1d5e2b,2019-11-01T00:56:50.000,Petición de un posible BOT,1,-233953730499517940,RTVE,Android,7.0,Chrome Mobile,76.0.3809,...,RADIO-3,Música,1,NaN,NaN,NaN,NaN,"90.0000, 0.0000",NaN,NaN


In [3]:
#Se eliminan las columnas con los datos que no son necesarios.
Datos = datos.drop(['id', 'value', 'os_version', 'browser_version', 'device_brand', 'device_model', 'episode_file_size',  'entries_processed', 'hostname', 'location', 'zip_code'], axis=1)

In [4]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid


In [5]:
Datos.shape

(10000, 17)

Las categorías recogidas en la columna 'metric' son las siguientes:

In [6]:
Datos['metric'].value_counts(dropna=False)

Número de descargas de tipo 206              3841
Petición de tipo HEAD                        3216
Número de descargas parciales de tipo 206    1262
Número de descargas parciales de tipo 200    1103
Petición no 200 o 206                         340
Número de descargas de tipo 200               204
Petición de un recurso no MP3                  17
Petición de un posible BOT                     17
Name: metric, dtype: int64

In [7]:
#Se eliminan los datos que no sean de tipo 206, 200 o descargas parciales.No son interesantes para el análisis.
Datos = Datos[Datos.metric.isin(['Número de descargas de tipo 206', 'Número de descargas parciales de tipo 206', 'Número de descargas parciales de tipo 200','Número de descargas de tipo 200' ])]

In [8]:
Datos.metric

0                 Número de descargas de tipo 206
1       Número de descargas parciales de tipo 200
2                 Número de descargas de tipo 206
3                 Número de descargas de tipo 206
5       Número de descargas parciales de tipo 206
                          ...                    
9994              Número de descargas de tipo 206
9995    Número de descargas parciales de tipo 206
9997    Número de descargas parciales de tipo 200
9998              Número de descargas de tipo 206
9999              Número de descargas de tipo 206
Name: metric, Length: 6410, dtype: object

In [9]:
Datos['metric'].value_counts(dropna=False)

Número de descargas de tipo 206              3841
Número de descargas parciales de tipo 206    1262
Número de descargas parciales de tipo 200    1103
Número de descargas de tipo 200               204
Name: metric, dtype: int64

In [10]:
Datos['referer'].value_counts(dropna=False)

NaN     3307
RTVE    3103
Name: referer, dtype: int64

In [11]:
#Se eliminan los datos cuya duración del episodio es 0.
Datos= Datos[Datos.episode_duration!=0]

In [12]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid


In [13]:
Datos['user'].value_counts(dropna=False)

-4651496039380642881    11
 496082641842722391     10
 6069712137940548831     6
 3413815924711635741     6
-480555356839012845      6
                        ..
 2057006852284194053     1
 8644027351722851357     1
 8589693044294506686     1
-7851554297868805363     1
 5733484840755179275     1
Name: user, Length: 4917, dtype: int64

In [14]:
Datos['podcast_permalink'].value_counts(dropna=False)

RADIO-NACIONAL    2418
RADIO-3           2177
RADIO-CLASICA      809
RADIO-5            271
RADIO-EXTERIOR     100
RADIO-4             21
NaN                 12
LA1                  1
Name: podcast_permalink, dtype: int64

In [15]:
Datos['podcast_title'].value_counts(dropna=False)

Las mañanas de RNE con Pepa Fernández    533
Espacio en blanco                        526
Las mañanas de RNE con Íñigo Alfonso     243
Hoy empieza todo con Ángel Carmona       210
180 grados                               206
                                        ... 
Diez minutos bien empleados                1
Vidas contadas                             1
Binaural                                   1
Libros para comprender                     1
Tablero deportivo                          1
Name: podcast_title, Length: 366, dtype: int64

In [16]:
#Ordenar los datos por franja horaria
Datos['hora'] = Datos.apply(lambda row: datetime.datetime.strptime(row.date, "%Y-%m-%dT%H:%M:%S.%f").hour , axis = 1)
Datos['minutos'] = Datos.apply(lambda row: datetime.datetime.strptime(row.date, "%Y-%m-%dT%H:%M:%S.%f").minute , axis = 1)

In [17]:
Datos.head(2)


,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone,hora,minutos
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid,0,56
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid,1,27


Clasificación de los datos por franja horaria.

    Mañana 7:00 a 15:00.
    Tarde 15:00 a 21:00.
    Noche 21:00 a 07 del día siguiente.
    


In [18]:
#Nueva columna en la matriz que indique a que franja horaria pertenece cada dato. 
Datos['franja_horaria']= Datos['hora'].apply(lambda x:'mañana' if (7 <= x < 13 ) else ('tarde' if (13 <= x < 21 )  else 'noche'))


In [19]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone,hora,minutos,franja_horaria
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid,0,56,noche
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,4674885,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid,1,27,noche


In [20]:
#Número de registros para cada franja horaria
Datos['franja_horaria'].value_counts()

mañana    2781
tarde     1569
noche     1459
Name: franja_horaria, dtype: int64

Función para filtrar los datos por duración.


Primero se clasifican los datos por duración. Están en milisegundos.
Se pasan los datos a segundos.

In [21]:
Datos['episode_duration']= Datos['episode_duration'].apply (lambda x: x/(60000))

In [22]:
Datos['episode_duration'].value_counts()

53.764533     149
118.894250    117
55.581783      97
53.356583      81
77.914750      75
             ... 
2.488717        1
118.980867      1
59.052917       1
59.097333       1
20.250000       1
Name: episode_duration, Length: 1921, dtype: int64

In [23]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,podcast_title,podcast_permalink,podcast_topic,city,region,country,time_zone,hora,minutos,franja_horaria
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,24 horas,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid,0,56,noche
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,24 horas,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid,1,27,noche


**Función que devuelva los datos según duración del episodio en minutos.**


Distinguimos varios tipos de podcasts:

    <30 mins.
    30-60 mins.
    >60 mins.

In [24]:
#Se añade una columna que clasifique los datos por duración:
Datos['clasificación_dur_ep']= Datos['episode_duration'].apply (lambda x: '<30 mins' if x<30 else('30-60 mins' if 30<=x<60 else '>60 mins'))

In [25]:
Datos.head(2)

,date,metric,user,referer,os_family,browser_family,device_family,episode_file_path,episode_title,episode_duration,...,podcast_permalink,podcast_topic,city,region,country,time_zone,hora,minutos,franja_horaria,clasificación_dur_ep
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,...,RADIO-NACIONAL,Informativos,Palencia,Castilla y León,España,Europe/Madrid,0,56,noche,>60 mins
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,NaN,Android,Android,Smartphone,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,...,RADIO-NACIONAL,Informativos,Madrid,Madrid,España,Europe/Madrid,1,27,noche,>60 mins


In [26]:
Datos.columns

Index(['date', 'metric', 'user', 'referer', 'os_family', 'browser_family',
       'device_family', 'episode_file_path', 'episode_title',
       'episode_duration', 'podcast_title', 'podcast_permalink',
       'podcast_topic', 'city', 'region', 'country', 'time_zone', 'hora',
       'minutos', 'franja_horaria', 'clasificación_dur_ep'],
      dtype='object')

In [27]:
Datos[['episode_duration','clasificación_dur_ep']].head()

,episode_duration,clasificación_dur_ep
0,77.914750,>60 mins
1,77.914750,>60 mins
2,58.553983,30-60 mins
3,53.356583,30-60 mins
5,53.786733,30-60 mins


In [28]:
Datos['podcast_permalink'].value_counts()

RADIO-NACIONAL    2418
RADIO-3           2177
RADIO-CLASICA      809
RADIO-5            271
RADIO-EXTERIOR     100
RADIO-4             21
LA1                  1
Name: podcast_permalink, dtype: int64

**Matriz de datos necesaria para el sistema de recomendación item-item**

In [29]:
Datos=Datos.drop(['referer', 'os_family', 'browser_family', 'device_family', 'time_zone', 'hora', 'minutos', 'podcast_permalink'], axis=1)

In [30]:
Datos.head(2)

,date,metric,user,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,city,region,country,franja_horaria,clasificación_dur_ep
0,2019-11-01T00:56:36.000,Número de descargas de tipo 206,6624207285322504776,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,24 horas,Informativos,Palencia,Castilla y León,España,noche,>60 mins
1,2019-11-01T01:27:37.000,Número de descargas parciales de tipo 200,1093782487219863411,1572564777864.mp3,Análisis de la información - 31/10/19,77.91475,24 horas,Informativos,Madrid,Madrid,España,noche,>60 mins


In [31]:
Datos.episode_file_path.value_counts(dropna=False)

1572174923017.mp3    149
1572511234660.mp3    117
1572179401007.mp3     97
1572175203168.mp3     81
1572564777864.mp3     75
                    ... 
1525449746498.mp3      1
1562803559138.mp3      1
1571383347660.mp3      1
1565606062892.mp3      1
1267728830372.mp3      1
Name: episode_file_path, Length: 1986, dtype: int64

In [32]:

descargas_usuario = dict()

In [33]:
descargas_usuario

{}

In [34]:
for row in Datos.itertuples(index=False):
    user = row[2]
    episode = row[3]
    if not user in descargas_usuario:
         descargas_usuario[user] = set()
    descargas_usuario[user].add(episode)

In [35]:
descargas_usuario

{6624207285322504776: {'1572564777864.mp3'},
 1093782487219863411: {'1572564777864.mp3'},
 -3042089131472286197: {'1572524596000.mp3'},
 -5995978442346962145: {'1572175203168.mp3'},
 -9171307944933687357: {'1564291777588.mp3', '1566107199700.mp3'},
 180705575781551235: {'1572464587664.mp3', '1572535417215.mp3'},
 8172620313194145685: {'1572552483655.mp3', '1572558201533.mp3'},
 -707564870363397593: {'1546621756302.mp3'},
 -1721142722692892358: {'1572513493553.mp3'},
 -3690287593443786727: {'1554615518145.mp3'},
 2821412558610718236: {'1572502971937.mp3'},
 -3410849907905118565: {'1572174923017.mp3'},
 5677672443816649724: {'1572564777864.mp3'},
 -7216757894137286627: {'1324056065181.mp3'},
 8547547893280042324: {'1563083619915.mp3', '1563685317681.mp3'},
 1754994420597413291: {'1566711681348.mp3'},
 -5540856614789222318: {'1572174923017.mp3'},
 -1842335803629375713: {'1572476431635.mp3'},
 -7829275662668923504: {'1307100410843.mp3', '1412330990047.mp3'},
 7062150472287768717: {'1572520

In [36]:
descargas_usuario[-4651496039380642881]

{'1537379824544.mp3',
 '1541071703720.mp3',
 '1542627318580.mp3',
 '1544616479193.mp3',
 '1559041754785.mp3',
 '1568585517517.mp3',
 '1570360932293.mp3',
 '1571509616605.mp3',
 '1571740911874.mp3',
 '1572259011860.mp3',
 '1572546538547.mp3'}

In [37]:
def pares_de_episodios(descargas_usuario):
    contador_episodio = dict()
    #key: un par de episodios y de valor un nº entero
    
    for episodes in descargas_usuario.values(): #iterar sobre las listas
        for subset in combinations(episodes, 2):
            
            par = frozenset([x for x in subset])   
            if len(par) == 2:
                if par in contador_episodio:
                    contador_episodio[par] = contador_episodio[par]+1
                else:
                    contador_episodio[par] = 1
    return contador_episodio

In [38]:
contador_episodio = pares_de_episodios(descargas_usuario)  


In [39]:
contador_episodio

{frozenset({'1564291777588.mp3', '1566107199700.mp3'}): 1,
 frozenset({'1572464587664.mp3', '1572535417215.mp3'}): 1,
 frozenset({'1572552483655.mp3', '1572558201533.mp3'}): 2,
 frozenset({'1563083619915.mp3', '1563685317681.mp3'}): 1,
 frozenset({'1307100410843.mp3', '1412330990047.mp3'}): 1,
 frozenset({'1572520854478.mp3', '1572524296652.mp3'}): 5,
 frozenset({'1572554317936.mp3', '1572564777864.mp3'}): 5,
 frozenset({'1572175203168.mp3', '1572179401007.mp3'}): 9,
 frozenset({'1559913753844.mp3', '1560950606977.mp3'}): 1,
 frozenset({'1571556023104.mp3', '1572175203168.mp3'}): 1,
 frozenset({'1572265104005.mp3', '1572348187503.mp3'}): 1,
 frozenset({'1508048483558.mp3', '1508654928933.mp3'}): 1,
 frozenset({'1551448381978.mp3', '1556893882519.mp3'}): 2,
 frozenset({'1572434639413.mp3', '1572438403249.mp3'}): 3,
 frozenset({'1572548238652.mp3', '1572550012941.mp3'}): 1,
 frozenset({'1559922657297.mp3', '1561732264396.mp3'}): 1,
 frozenset({'1561120377237.mp3', '1561732264396.mp3'}): 

In [40]:
len(contador_episodio)

1137

In [41]:
episodes_combination = open("tabla_episodios","w")
#Serialize
episodes_combination.write("Episodio1;Episodio2;Rating\n")
for key in contador_episodio.keys():
    for element in key:
        episodes_combination.write(element+";")
    episodes_combination.write(str(contador_episodio[key]))
    episodes_combination.write("\n")
episodes_combination.close()

In [42]:
contador_episodio[key]

1

In [43]:
episodes_combination

<_io.TextIOWrapper name='tabla_episodios' mode='w' encoding='cp1252'>

In [44]:
episodes_rating= pd.read_csv('tabla_episodios', delimiter=";")

In [45]:
episodes_rating

,Episodio1,Episodio2,Rating
0,1566107199700.mp3,1564291777588.mp3,1
1,1572464587664.mp3,1572535417215.mp3,1
2,1572558201533.mp3,1572552483655.mp3,2
3,1563685317681.mp3,1563083619915.mp3,1
4,1412330990047.mp3,1307100410843.mp3,1
...,...,...,...
1132,1535814586615.mp3,1536420230201.mp3,1
1133,1571851245135.mp3,1571764836181.mp3,1
1134,1560012907660.mp3,1555929546019.mp3,1
1135,1571391531949.mp3,1572604988093.mp3,1


In [46]:
episodes_rating= episodes_rating.sort_values(by='Rating', ascending=False)
episodes_rating

,Episodio1,Episodio2,Rating
27,1572174923017.mp3,1572179401007.mp3,12
7,1572175203168.mp3,1572179401007.mp3,9
110,1572520854478.mp3,1572518833245.mp3,6
30,1572564777864.mp3,1572557657655.mp3,6
176,1572521397366.mp3,1572511234660.mp3,5
...,...,...,...
410,1572434499678.mp3,1572038588014.mp3,1
411,1572604988093.mp3,1572600025093.mp3,1
412,1572464587664.mp3,1571422920620.mp3,1
413,1571232447064.mp3,1572358990299.mp3,1


In [47]:
episodes_rating.columns

Index(['Episodio1', 'Episodio2', 'Rating'], dtype='object')

**Matriz csr_ sparse X**

Generar una matriz sparse para la tabla de datos episodes_rating

In [48]:
from scipy.sparse import csr_matrix

def crear_X(episodes_rating):
    """
    Genera una matriz sparse para los datos de la tabla episode_rating.
    
    Args:
        episodes_rating: pandas dataframe
    
    Returns:
        X: sparse matrix
        episode1_mapper: dict que relaciona el episode_file_path con un indice.
       ep1_inv_mapper: dict que relaciona el índice con el file del episodio1.
        episode2_mapper: dict que relaciona el episode_file_path (columna 2) con un indice.
        ep2_inv_mapper: dict que relaciona el índice con el file del episodio2.
    """
    N = episodes_rating['Episodio1'].nunique()
    M = episodes_rating['Episodio2'].nunique()

    episode1_mapper = dict(zip(np.unique(episodes_rating["Episodio1"]), list(range(N))))
    episode2_mapper = dict(zip(np.unique(episodes_rating["Episodio2"]), list(range(M))))
    
    ep1_inv_mapper = dict(zip(list(range(N)), np.unique(episodes_rating["Episodio1"])))
    ep2_inv_mapper = dict(zip(list(range(M)), np.unique(episodes_rating["Episodio2"])))
    
    ep1_index = [episode1_mapper[i] for i in episodes_rating['Episodio1']]
    ep2_index = [episode2_mapper[i] for i in episodes_rating['Episodio2']]

    X = csr_matrix((episodes_rating["Rating"], (ep2_index, ep1_index)), shape=(M, N))
    
    return X, episode1_mapper, episode2_mapper, ep1_inv_mapper, ep2_inv_mapper

In [49]:
episodes_rating['Episodio1'].nunique()

568

In [50]:
episodes_rating['Episodio2'].nunique()

560

In [51]:
X, episode1_mapper, episode2_mapper, ep1_inv_mapper, ep2_inv_mapper = crear_X(episodes_rating)

In [143]:
episode1_mapper

{'1228500668293.mp3': 0,
 '1229423025184.mp3': 1,
 '1239721903454.mp3': 2,
 '1248091164548.mp3': 3,
 '1255422807594.mp3': 4,
 '1256554098683.mp3': 5,
 '1257992281351.mp3': 6,
 '1261016292377.mp3': 7,
 '1262163524864.mp3': 8,
 '1267433098059.mp3': 9,
 '1273871089739.mp3': 10,
 '1274710880836.mp3': 11,
 '1281523936036.mp3': 12,
 '1287753893941.mp3': 13,
 '1295021604079.mp3': 14,
 '1308322456442.mp3': 15,
 '1311942506599.mp3': 16,
 '1317540652437.mp3': 17,
 '1318745918404.mp3': 18,
 '1347091500517.mp3': 19,
 '1370254809382.mp3': 20,
 '1375107873818.mp3': 21,
 '1389355891976.mp3': 22,
 '1402054540200.mp3': 23,
 '1412330990047.mp3': 24,
 '1413131999898.mp3': 25,
 '1422109319001.mp3': 26,
 '1426860343611.mp3': 27,
 '1428949108003.mp3': 28,
 '1429865890389.mp3': 29,
 '1432364276751.mp3': 30,
 '1433847641792.mp3': 31,
 '1434699700900.mp3': 32,
 '1435235679607.mp3': 33,
 '1435833085951.mp3': 34,
 '1436343524855.mp3': 35,
 '1436686563114.mp3': 36,
 '1437479564370.mp3': 37,
 '1437484778378.mp3': 

In [52]:
episodes_rating[episodes_rating.Episodio1=='1572175203168.mp3']

,Episodio1,Episodio2,Rating
7,1572175203168.mp3,1572179401007.mp3,9
552,1572175203168.mp3,1572174923017.mp3,3
751,1572175203168.mp3,1570944650194.mp3,1
831,1572175203168.mp3,1572513493553.mp3,1
897,1572175203168.mp3,1568524952004.mp3,1
9,1572175203168.mp3,1571556023104.mp3,1


In [53]:
episodes_rating[episodes_rating.Episodio2=='1572175203168.mp3']

,Episodio1,Episodio2,Rating


In [54]:
from scipy.sparse import save_npz

save_npz('item_item_matrix.npz', X)

In [55]:
from sklearn.neighbors import NearestNeighbors

def similar_episodes(episode_file, X, k, metric='cosine', show_distance=False):
    """
    Finds k-nearest neighbours for a given episode_file.
    
    Args:
        episode_file: file del episodio sbore el que se recomienda
        X: item-item utility matrix
        k: número de episodios similares
        metric: métrica para los kNN cálculos
    
    Returns:
        lista de k episodios similares
    """
    neighbour_ids = []
    
    
    if episode_file in episode2_mapper:
     episode_ind=episode2_mapper[episode_file]
    
    else:
     episode_ind=episode1_mapper[episode_file]

    
    
    episode_vec = X[episode_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    if isinstance(episode_vec, (np.ndarray)):
        episode_vec = episode_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(episode_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(ep2_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [56]:
sparsity = X.count_nonzero()/(X.shape[0]*X.shape[1])

print(f"Matrix sparsity: {round(sparsity*100,2)}%")

Matrix sparsity: 0.36%


In [57]:
episodes_rating.iloc[183]

Episodio1    1559041754785.mp3
Episodio2    1572259011860.mp3
Rating                       1
Name: 646, dtype: object

In [58]:
tabla_ref= Datos[['episode_file_path', 'episode_title', 'episode_duration', 'podcast_title', 'podcast_topic']]
tabla_ref=tabla_ref.drop_duplicates()


In [59]:
tabla_ref

,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic
0,1572564777864.mp3,Análisis de la información - 31/10/19,77.914750,24 horas,Informativos
2,1572524596000.mp3,A través de un espejo - 31/10/19,58.553983,A través de un espejo,Música
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento
5,1564291777588.mp3,El pueblo de los avistamientos,53.786733,Espacio en blanco,Entretenimiento
6,1572535417215.mp3,Música de Stevie Wonder,59.141750,Cuando los elefantes sueñan con la música,Música
...,...,...,...,...,...
9964,1569303678565.mp3,Todos somos sospechosos - Combate: Ficción vs....,59.557100,Todos somos sospechosos,Música
9977,1519411919356.mp3,Viernes eléctrico (257),58.558833,Turbo 3,Música
9992,1525449746498.mp3,El niño al que se le murió el amigo (Ana María...,2.943233,LiterCast,RNE solo en Podcast
9994,1473625395534.mp3,Pajaritos y pajarracos,60.093617,Sonideros,Música


In [60]:
dic_episodios = dict(zip(tabla_ref.episode_file_path,tabla_ref.episode_title))

In [61]:
#Función de sistema de recomendación de episodios

def recomendar_episodios(episodio):
    lista = []  
    dic_recom = dict()
 

    print("Recomendación para:")
    print(dic_episodios[episodio])
    
   
    if not episodio in (episode1_mapper or episode2_mapper): 
        print('\nNo hay recomendaciones para este episodio')
        
    else:     
        similar_ids = similar_episodes(episodio, X, k=5)
    
        #print(similar_ids)
        print("\n")

        for i in similar_ids:
          # print(dic_episodios[i], "," , i )
            dic_recom[i]= dic_episodios[i]
            tabla_recom = pd.DataFrame([[key, dic_recom[key]] for key in dic_recom.keys()], columns=['episode_file_path', 'episode_title'])
       
     
            for file in tabla_recom.episode_file_path:
                lista.append(file)
        
    df_top_recom = tabla_ref[tabla_ref.episode_file_path==episodio].append(tabla_ref[tabla_ref.episode_file_path.isin(lista)])
    
   
    
    return df_top_recom
      

In [62]:
recomendar_episodios('1572535417215.mp3')

Recomendación para:
Música de Stevie Wonder




,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic
6,1572535417215.mp3,Música de Stevie Wonder,59.141750,Cuando los elefantes sueñan con la música,Música
158,1572287098135.mp3,"Tame Impala, Foals, Colectivo Da Silva y Idles...",58.963667,Turbo 3,Música
558,1572521397366.mp3,Fuel Fandango y Haim,59.184850,180 grados,Música
864,1572463146122.mp3,Nunatak y Taylor Hawkins & The Coattail Riders,58.887050,Turbo 3,Música
1291,1572024178893.mp3,Viernes eléctrico (338),59.021567,Turbo 3,Música
1463,1572434499678.mp3,"Algiers, Torres y Ginebras en directo",58.952800,180 grados,Música


In [63]:
recomendar_episodios('1572175203168.mp3')

Recomendación para:
Encuentros con la vida después de la vida




,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento
295,1571824405362.mp3,"Ravel, Borodin, Scriabin y Elgar",89.095050,Música a la carta,Música
1816,1572172900111.mp3,Tonalidades,26.726400,El rincón de la teoría,Música
1824,1570347772665.mp3,La mirada de Van Gogh,59.003300,Café del sur,Música
9570,1546762406225.mp3,Génova,58.888400,Café del sur,Música


In [64]:
duración = dict(zip(tabla_ref.episode_file_path,tabla_ref.episode_duration))


In [65]:
#Función que devuelva grado de pertenencia según duración
#Función de pertenencia triangular

def función_triang_pert (episodio, porcentaje):
    
   df_ranking= recomendar_episodios(episodio)
   porcentaje = porcentaje/100
   a = (duración[episodio]) - (duración[episodio]*porcentaje)
   m = (duración[episodio])
   b = (duración[episodio]) + (duración[episodio]*porcentaje)
   #print("Límite inferior:", round(a,2))
   #print("Moda", round(m,2))
   #print("Límite superior:", round(b,2), "\n")
    
   
    
   df_ranking['pert_duración']= df_ranking['episode_duration'].apply(lambda x: 1 if x == m else (0 if x <= a else (((x-a)/(m-a)) if a < x < m else (((b-x)/(b-m)) if m < x < b else 0))))
    
            
   return df_ranking

In [66]:
función_triang_pert('1572175203168.mp3', 20)

Recomendación para:
Encuentros con la vida después de la vida




,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento,1.000000
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,0.961771
295,1571824405362.mp3,"Ravel, Borodin, Scriabin y Elgar",89.095050,Música a la carta,Música,0.000000
1816,1572172900111.mp3,Tonalidades,26.726400,El rincón de la teoría,Música,0.000000
1824,1570347772665.mp3,La mirada de Van Gogh,59.003300,Café del sur,Música,0.470851
9570,1546762406225.mp3,Génova,58.888400,Café del sur,Música,0.481618


**Cálculo de la función de pertenencia según temática**

A continuación, se genera una tabla que calcule el número de veces que cada par de temáticas han sido escuchadas conjuntamente.

In [67]:
topics_usuario = dict()

In [68]:
topics_usuario

{}

In [69]:
for row in Datos.itertuples(index=False):
    user = row[2]
    topic = row[7]
    if not user in topics_usuario:
         topics_usuario[user] = set()
    topics_usuario[user].add(topic)

In [70]:
topics_usuario[-4651496039380642881]

{'Ciencia y Tecnología',
 'Cultura',
 'Entretenimiento',
 'Informativos',
 'Viajes'}

In [71]:
def pares_de_topics(topics_usuario):
    contador_topic = dict()
    #key: un par de episodios y de valor un nº entero
    
    for topic in topics_usuario.values(): #iterar sobre las listas
        for subset in combinations(topic, 2):
            
            par = frozenset([x for x in subset])   
            if len(par) == 2:
                if par in contador_topic:
                    contador_topic[par] = contador_topic[par]+1
                else:
                    contador_topic[par] = 1
    return contador_topic

In [72]:
contador_topic = pares_de_topics(topics_usuario)  


In [73]:
contador_topic

{frozenset({'Informativos', 'Música'}): 7,
 frozenset({'Documentales', 'Entretenimiento'}): 2,
 frozenset({'Cultura', 'Educación'}): 1,
 frozenset({'Magacín', 'Música'}): 9,
 frozenset({'Entretenimiento', 'Viajes'}): 2,
 frozenset({'Entretenimiento', 'Magacín'}): 6,
 frozenset({'Documentales', 'Magacín'}): 2,
 frozenset({'Archivo RNE', 'Servicio público'}): 1,
 frozenset({'Informativos', 'Magacín'}): 8,
 frozenset({'Entretenimiento', 'Informativos'}): 5,
 frozenset({'Documentales', 'Música'}): 1,
 frozenset({'Archivo RNE', 'Informativos'}): 3,
 frozenset({'Ciencia y Tecnología', 'Magacín'}): 1,
 frozenset({'Cultura', 'Magacín'}): 3,
 frozenset({'Informativos territoriales', 'Música'}): 1,
 frozenset({'Educación', 'Infantiles'}): 1,
 frozenset({'Ciencia y Tecnología', 'Informativos'}): 2,
 frozenset({'Archivo RNE', 'Música'}): 1,
 frozenset({'Informativos', 'Informativos territoriales'}): 2,
 frozenset({'Ciencia y Tecnología', 'Cultura'}): 1,
 frozenset({'Cultura', 'Viajes'}): 1,
 froze

In [74]:
topics_combination = open("tabla_topics","w")
#Serialize
topics_combination.write("Topic1;Topic2;Puntuacion\n")
for key in contador_topic.keys():
    for element in key:
        topics_combination.write(element+";")
    topics_combination.write(str(contador_topic[key]))
    topics_combination.write("\n")
topics_combination.close()

In [75]:
topics_combination

<_io.TextIOWrapper name='tabla_topics' mode='w' encoding='cp1252'>

In [76]:
topics_rating= pd.read_csv('tabla_topics', delimiter=";", encoding='latin-1')

In [77]:
topics_rating=topics_rating.sort_values(by='Puntuacion', ascending=False)

In [78]:
topics_rating

,Topic1,Topic2,Puntuacion
3,Magacín,Música,9
8,Magacín,Informativos,8
0,Música,Informativos,7
5,Magacín,Entretenimiento,6
9,Entretenimiento,Informativos,5
21,Cultura,Informativos,4
34,Magacín,Servicio público,3
11,Archivo RNE,Informativos,3
13,Magacín,Cultura,3
4,Entretenimiento,Viajes,2


In [79]:
from scipy.stats import norm
import statistics as stats

In [80]:
#Función que determine el peso entre las parejas de temas para cada Topic propuesto
def prob_topic(topic):
 df_prob_topic = topics_rating[topics_rating.Topic1==topic].append(topics_rating[topics_rating.Topic2==topic]).sort_values(by='Puntuacion', ascending=False)
 suma_total=df_prob_topic.Puntuacion.sum()
 df_prob_topic['Probabilidad']= df_prob_topic['Puntuacion'].apply(lambda x: x/suma_total)
 df_prob_topic['CDF']= norm.cdf(df_prob_topic['Puntuacion'], loc=stats.mean(df_prob_topic['Puntuacion']), scale=stats.stdev(df_prob_topic['Puntuacion']))

 return (df_prob_topic)

In [81]:
prob_topic('Música')

,Topic1,Topic2,Puntuacion,Probabilidad,CDF
3,Magacín,Música,9,0.409091,0.971485
0,Música,Informativos,7,0.318182,0.902183
36,Música,Cultura,1,0.045455,0.297065
17,Música,Archivo RNE,1,0.045455,0.297065
10,Música,Documentales,1,0.045455,0.297065
32,Entretenimiento,Música,1,0.045455,0.297065
27,RNE solo en Podcast,Música,1,0.045455,0.297065
14,Informativos territoriales,Música,1,0.045455,0.297065


In [82]:
for row in prob_topic('Música').itertuples(index=False):
    if 'Entretenimiento' in row:
        print(row.Probabilidad)

0.045454545454545456


In [83]:
ej= list(tabla_ref['podcast_topic'][tabla_ref.episode_file_path=='1572175203168.mp3'])

In [84]:
ej

['Entretenimiento']

In [85]:
ej_musica= prob_topic('Música')

In [86]:
ej_musica

,Topic1,Topic2,Puntuacion,Probabilidad,CDF
3,Magacín,Música,9,0.409091,0.971485
0,Música,Informativos,7,0.318182,0.902183
36,Música,Cultura,1,0.045455,0.297065
17,Música,Archivo RNE,1,0.045455,0.297065
10,Música,Documentales,1,0.045455,0.297065
32,Entretenimiento,Música,1,0.045455,0.297065
27,RNE solo en Podcast,Música,1,0.045455,0.297065
14,Informativos territoriales,Música,1,0.045455,0.297065


In [87]:
ej_musica['suma']=ej_musica[['Probabilidad', 'CDF']].sum(axis=1)


In [88]:
ej_musica

,Topic1,Topic2,Puntuacion,Probabilidad,CDF,suma
3,Magacín,Música,9,0.409091,0.971485,1.380575
0,Música,Informativos,7,0.318182,0.902183,1.220365
36,Música,Cultura,1,0.045455,0.297065,0.342520
17,Música,Archivo RNE,1,0.045455,0.297065,0.342520
10,Música,Documentales,1,0.045455,0.297065,0.342520
32,Entretenimiento,Música,1,0.045455,0.297065,0.342520
27,RNE solo en Podcast,Música,1,0.045455,0.297065,0.342520
14,Informativos territoriales,Música,1,0.045455,0.297065,0.342520


In [89]:
pru=dict()

In [90]:
for row in Datos.itertuples(index=False):
    user = row[2]
    episode = row[3]
    if not user in descargas_usuario:
         descargas_usuario[user] = set()
    descargas_usuario[user].add(episode)

In [91]:
def tabla_pert_2 (episodio, porcentaje):
    lista_prob_topics = dict()
    df_pert= función_triang_pert (episodio, porcentaje)
    tema = df_pert.iloc[0]['podcast_topic']
    df_prob_tema = prob_topic(tema)
    #print (tema)
    print("\n")
    
    for row in df_prob_tema.itertuples(index=False):
   
       if row[0]==tema:
               topic = row[1]
       else:
               topic= row[0]
            
       prob = row[3]
       
       if not topic in lista_prob_topics:
               lista_prob_topics[topic] = set()
       lista_prob_topics[topic]=prob
    
    
    #print(lista_prob_topics)
    
   
   
        
        
    df_pert['pert_tema']= df_pert['podcast_topic'].apply(lambda x: 1 if x == tema else lista_prob_topics[x]) 
    
    
    return   df_pert #df_prob_tema


In [92]:
tabla_pert_2('1572175203168.mp3', 20)

Recomendación para:
Encuentros con la vida después de la vida






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento,1.000000,1.000000
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,0.961771,1.000000
295,1571824405362.mp3,"Ravel, Borodin, Scriabin y Elgar",89.095050,Música a la carta,Música,0.000000,0.052632
1816,1572172900111.mp3,Tonalidades,26.726400,El rincón de la teoría,Música,0.000000,0.052632
1824,1570347772665.mp3,La mirada de Van Gogh,59.003300,Café del sur,Música,0.470851,0.052632
9570,1546762406225.mp3,Génova,58.888400,Café del sur,Música,0.481618,0.052632


In [93]:
tabla_pert_2('1572564777864.mp3', 20)

Recomendación para:
Análisis de la información - 31/10/19 






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema
0,1572564777864.mp3,Análisis de la información - 31/10/19,77.914750,24 horas,Informativos,1.000000,1.000000
863,1572164781354.mp3,El pequeño salvaje,59.091233,Café del sur,Música,0.000000,0.212121
1250,1572558201533.mp3,¿Cómo se fraguó el proyecto de República digit...,2.910900,24 horas,Informativos,0.000000,1.000000
1460,1572560588519.mp3,"""Programas informáticos que suplantan las voce...",9.347033,24 horas,Informativos,0.000000,1.000000
1566,1572472802536.mp3,"En algún lugar del mundo: ""El camino del Brexit""",10.741550,24 horas,Informativos,0.000000,1.000000
5692,1572388932885.mp3,Análisis de la información - 29/10/19,67.844550,24 horas,Informativos,0.353768,1.000000


In [94]:
prob_topic('Entretenimiento')

,Topic1,Topic2,Puntuacion,Probabilidad,CDF
5,Magacín,Entretenimiento,6,0.315789,0.951821
9,Entretenimiento,Informativos,5,0.263158,0.876305
4,Entretenimiento,Viajes,2,0.105263,0.358873
22,Entretenimiento,Cultura,2,0.105263,0.358873
1,Entretenimiento,Documentales,2,0.105263,0.358873
32,Entretenimiento,Música,1,0.052632,0.192825
24,Ciencia y Tecnología,Entretenimiento,1,0.052632,0.192825


Se establece una nueva columna de puntuaciones siguiendo una función de distribución acumulada

In [95]:
def tabla_pert_3 (episodio, porcentaje):
    lista_prob_topics = dict()
    df_pert= función_triang_pert (episodio, porcentaje)
    tema = df_pert.iloc[0]['podcast_topic']
    df_prob_tema = prob_topic(tema)
    #print (tema)
    print("\n")
    
    for row in df_prob_tema.itertuples(index=False):
   
       if row[0]==tema:
               topic = row[1]
       else:
               topic= row[0]
            
       cdf = row[4]
       
       if not topic in lista_prob_topics:
               lista_prob_topics[topic] = set()
       lista_prob_topics[topic]=cdf
    
    
    #print(lista_prob_topics)
    
        
    df_pert['pert_tema']= df_pert['podcast_topic'].apply(lambda x: 1 if x == tema else (lista_prob_topics[x] if x in lista_prob_topics else 0)) 
    df_pert['suma']=df_pert[['pert_duración', 'pert_tema']].sum(axis=1)
    df_pert['%']=df_pert['suma'].apply(lambda x: round((x/2)*100, 2))
    df_pert=df_pert.drop(['suma'], axis=1)
    
    return   df_pert #df_prob_tema

In [96]:
tabla_pert_3('1572175203168.mp3', 20)

Recomendación para:
Encuentros con la vida después de la vida






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento,1.000000,1.000000,100.00
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,0.961771,1.000000,98.09
295,1571824405362.mp3,"Ravel, Borodin, Scriabin y Elgar",89.095050,Música a la carta,Música,0.000000,0.192825,9.64
1816,1572172900111.mp3,Tonalidades,26.726400,El rincón de la teoría,Música,0.000000,0.192825,9.64
1824,1570347772665.mp3,La mirada de Van Gogh,59.003300,Café del sur,Música,0.470851,0.192825,33.18
9570,1546762406225.mp3,Génova,58.888400,Café del sur,Música,0.481618,0.192825,33.72


In [97]:
tabla_pert_3('1572524296652.mp3', 20)

Recomendación para:
Tercera hora - 31/10/19






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
31,1572524296652.mp3,Tercera hora - 31/10/19,52.906850,Las mañanas de RNE con Pepa Fernández,Magacín,1.000000,1.000000,100.00
96,1572434002527.mp3,"Eduardo Mendoza: ""La perplejidad está a punto ...",30.422850,Las mañanas de RNE con Pepa Fernández,Magacín,0.000000,1.000000,50.00
433,1572038588014.mp3,Fracasología,119.536867,Mundo Babel,Música,0.000000,0.939926,47.00
2346,1572264713963.mp3,Operando - Turandot... más allá de Nessun Dorma,26.056033,Las mañanas de RNE con Pepa Fernández,Magacín,0.000000,1.000000,50.00
2465,1571995671979.mp3,Primera hora - 25/10/19,53.231633,Las mañanas de RNE con Pepa Fernández,Magacín,0.969306,1.000000,98.47
3864,1572529515698.mp3,"Susana Peris & Ramón Leal, Jack Bosco",29.684883,English Language Broadcast,Magacín,0.000000,1.000000,50.00


In [98]:
def Ranking_final (episode, porcentaje):
    tabla= tabla_pert_3(episode, porcentaje)
    tabla= tabla.sort_values(by='%', ascending=False)

 
        
    return tabla

In [99]:
Ranking_final ('1572524296652.mp3', 20)

Recomendación para:
Tercera hora - 31/10/19






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
31,1572524296652.mp3,Tercera hora - 31/10/19,52.906850,Las mañanas de RNE con Pepa Fernández,Magacín,1.000000,1.000000,100.00
2465,1571995671979.mp3,Primera hora - 25/10/19,53.231633,Las mañanas de RNE con Pepa Fernández,Magacín,0.969306,1.000000,98.47
96,1572434002527.mp3,"Eduardo Mendoza: ""La perplejidad está a punto ...",30.422850,Las mañanas de RNE con Pepa Fernández,Magacín,0.000000,1.000000,50.00
2346,1572264713963.mp3,Operando - Turandot... más allá de Nessun Dorma,26.056033,Las mañanas de RNE con Pepa Fernández,Magacín,0.000000,1.000000,50.00
3864,1572529515698.mp3,"Susana Peris & Ramón Leal, Jack Bosco",29.684883,English Language Broadcast,Magacín,0.000000,1.000000,50.00
433,1572038588014.mp3,Fracasología,119.536867,Mundo Babel,Música,0.000000,0.939926,47.00


In [100]:
Ranking_final ('1572175203168.mp3', 20)

Recomendación para:
Encuentros con la vida después de la vida






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento,1.000000,1.000000,100.00
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,0.961771,1.000000,98.09
9570,1546762406225.mp3,Génova,58.888400,Café del sur,Música,0.481618,0.192825,33.72
1824,1570347772665.mp3,La mirada de Van Gogh,59.003300,Café del sur,Música,0.470851,0.192825,33.18
295,1571824405362.mp3,"Ravel, Borodin, Scriabin y Elgar",89.095050,Música a la carta,Música,0.000000,0.192825,9.64
1816,1572172900111.mp3,Tonalidades,26.726400,El rincón de la teoría,Música,0.000000,0.192825,9.64


In [101]:
Ranking_final('1572564777864.mp3', 40)

Recomendación para:
Análisis de la información - 31/10/19 






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
0,1572564777864.mp3,Análisis de la información - 31/10/19,77.914750,24 horas,Informativos,1.000000,1.000000,100.00
5692,1572388932885.mp3,Análisis de la información - 29/10/19,67.844550,24 horas,Informativos,0.676884,1.000000,83.84
863,1572164781354.mp3,El pequeño salvaje,59.091233,Café del sur,Música,0.396022,0.904468,65.02
1250,1572558201533.mp3,¿Cómo se fraguó el proyecto de República digit...,2.910900,24 horas,Informativos,0.000000,1.000000,50.00
1460,1572560588519.mp3,"""Programas informáticos que suplantan las voce...",9.347033,24 horas,Informativos,0.000000,1.000000,50.00
1566,1572472802536.mp3,"En algún lugar del mundo: ""El camino del Brexit""",10.741550,24 horas,Informativos,0.000000,1.000000,50.00


In [102]:
Ranking_final('1570795278157.mp3', 30)

Recomendación para:
Especial MOZART






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
974,1570795278157.mp3,Especial MOZART,60.953383,Música y significado,Música,1.000000,1,100.00
1437,1569968973185.mp3,El retorno,60.311583,Música antigua,Música,0.964902,1,98.25
1410,1571178471565.mp3,La música en los monasterios,60.085650,Música antigua,Música,0.952547,1,97.63
93,1572447584727.mp3,La fantasía,60.024250,Músicas con alma,Música,0.949189,1,97.46
2454,1572276437030.mp3,Mozart: música para banquetes y jardines,57.946200,La hora azul,Música,0.835547,1,91.78
2125,1571782713973.mp3,Episodios de la vida de Bach,57.903533,Música antigua,Música,0.833214,1,91.66


**EVALUACIÓN DE RESULTADOS MEDIANTE PRUEBAS**

Para un episodio en particular, se valora antes y después de aplicar el Ranking

In [103]:
recomendar_episodios('1572174923017.mp3')

Recomendación para:
El Houdini de la Inquisición




,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento
16,1572513493553.mp3,Cuarta hora - 31/10/19,60.105317,Las mañanas de RNE con Íñigo Alfonso,Informativos
45,1572179401007.mp3,El hombre lobo de Yeste,55.581783,Espacio en blanco,Entretenimiento
79,1571556023104.mp3,El valle de la muerte,53.785433,Espacio en blanco,Entretenimiento
282,1570944650194.mp3,Presencias inexplicables,53.714900,Espacio en blanco,Entretenimiento
2618,1568524952004.mp3,"Dioses, reinas, templos y otros enigmas de Egipto",53.637400,Espacio en blanco,Entretenimiento


In [104]:
Ranking_final('1572174923017.mp3', 25)

Recomendación para:
El Houdini de la Inquisición






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,1.000000,1.000000,100.00
79,1571556023104.mp3,El valle de la muerte,53.785433,Espacio en blanco,Entretenimiento,0.998445,1.000000,99.92
282,1570944650194.mp3,Presencias inexplicables,53.714900,Espacio en blanco,Entretenimiento,0.996307,1.000000,99.82
2618,1568524952004.mp3,"Dioses, reinas, templos y otros enigmas de Egipto",53.637400,Espacio en blanco,Entretenimiento,0.990541,1.000000,99.53
45,1572179401007.mp3,El hombre lobo de Yeste,55.581783,Espacio en blanco,Entretenimiento,0.864799,1.000000,93.24
16,1572513493553.mp3,Cuarta hora - 31/10/19,60.105317,Las mañanas de RNE con Íñigo Alfonso,Informativos,0.528255,0.876305,70.23


In [105]:
Ranking_final('1572590227569.mp3', 30)

Recomendación para:
Placer y sobredosis musical 






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
3480,1572590227569.mp3,Placer y sobredosis musical,60.283300,A lo loco y con patines,Música,1,1.000000,100.00
2561,1572550634193.mp3,Noticies Vespre Balears 31/10/2019,9.862083,Radiodiari Migdia - Balears,Informativos territoriales,0,0.297065,14.85
2702,1571989268445.mp3,Notícies 9:15 Balears - 25/10/19,15.059150,Radiodiari Migdia - Balears,Informativos territoriales,0,0.297065,14.85
2815,1572552754883.mp3,Informativo Asturias Tarde 31/10/19,9.930883,Informativo de Asturias,Informativos territoriales,0,0.297065,14.85
3043,1572550719074.mp3,Informativo tarde clm 31/10/19,9.889200,Informativo de Castilla-La Mancha,Informativos territoriales,0,0.297065,14.85
3131,1572419406229.mp3,"Informativo País Vasco 07,45 - 30/10/19",14.966100,Besaide-País Vasco,Informativos territoriales,0,0.297065,14.85


Cambiando la métrica de recomendación:

In [115]:
Ranking_final('1572174923017.mp3', 25)

Recomendación para:
El Houdini de la Inquisición






,episode_file_path,episode_title,episode_duration,podcast_title,podcast_topic,pert_duración,pert_tema,%
19,1572174923017.mp3,El Houdini de la Inquisición,53.764533,Espacio en blanco,Entretenimiento,1.000000,1.000000,100.00
2618,1568524952004.mp3,"Dioses, reinas, templos y otros enigmas de Egipto",53.637400,Espacio en blanco,Entretenimiento,0.990541,1.000000,99.53
5715,1564899505120.mp3,El tesoro de los reyes,54.086267,Espacio en blanco,Entretenimiento,0.976064,1.000000,98.80
3,1572175203168.mp3,Encuentros con la vida después de la vida,53.356583,Espacio en blanco,Entretenimiento,0.969649,1.000000,98.48
2637,1572506351001.mp3,Segunda hora - 31/10/19,60.095300,Las mañanas de RNE con Íñigo Alfonso,Informativos,0.529001,0.876305,70.27
5236,1466767691288.mp3,"Xavier Cugat, el sueño americano a ritmo latino",55.601083,Documentos RNE,Documentales,0.863363,0.358873,61.11


**PRUEBAS CON 20 MUESTRAS**